# Summary Rport part 2

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima.model import ARIMA

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from scripts.utils import ts_conversion

## Modeling approaches

For these time-series forecasting business problem, two approaches will be taken:

- a supervised approach, using the Random Forest Regressor.

- an unsupervised approach, using an AutoRegressive Integrated Moving Average (ARIMA) Model.

## Modeling

In [2]:
original_df = pd.read_csv("./data/df_training.csv")

In [3]:
df = ts_conversion(original_df)

In [4]:
missing_dates = set(pd.date_range(df.index.min(), df.index.max())).difference(set(df.index))
len(missing_dates)

116

In [6]:
# linear interpolation for missing dates
resampled_df = df.resample('D').mean().interpolate()

In [7]:
# training set 80%, test set 20%
training_perc = 0.8

first_date_training = resampled_df.index.min()
last_date_training = first_date_training + pd.Timedelta(days=int(len(resampled_df)*training_perc))
first_date_testing = last_date_training + pd.Timedelta(days=1)
last_date_testing = resampled_df.index.max()

In [8]:
first_date_training, last_date_training, first_date_testing, last_date_testing

(Timestamp('2017-11-28 00:00:00', freq='D'),
 Timestamp('2019-03-31 00:00:00', freq='D'),
 Timestamp('2019-04-01 00:00:00', freq='D'),
 Timestamp('2019-07-31 00:00:00', freq='D'))

In [14]:
ss = StandardScaler()
mms = MinMaxScaler()

In [15]:
sscaled_resampled_df = pd.DataFrame(ss.fit_transform(resampled_df), 
                         index=resampled_df.index, 
                         columns=resampled_df.columns)

In [ ]:
ss.inverse_transform(sscaled_resampled_df)

In [ ]:
sscaled_resampled_df.loc[first_date_training:last_date_training], 

In [21]:
training_data = sscaled_resampled_df.loc[first_date_training:last_date_training, :]
testing_data = sscaled_resampled_df.loc[first_date_testing:last_date_testing, :]

In [27]:
def add_supervised_target(df, hm_days):
    result_df = df.copy()

    target = []

    for day in result_df.index:

        start = day + pd.Timedelta(days=1)
        end = start + pd.Timedelta(days=hm_days)

        rev_next_days = df["revenue"].loc[start:end].sum()

        target.append(rev_next_days)

    result_df["target"] = target

    return result_df

In [23]:
def convert_to_supervised(df, hm_days=30, functions=("mean",), day_windows=(3, 5, 7)):
    
    df_with_target = add_supervised_target(df, hm_days)
    
    df_with_new_variables = add_supervised_variables(df, variables, functions, day_windows)
    
    supervised_df = pd.merge(df_with_target, df_with_new_variables, on="date")
    
    return supervised_df

In [71]:
def add_supervised_variables(df, variables, functions=("mean",), day_windows=(3, 5, 7)):
    
    func_names = {"mean": np.mean, "std": np.std, "var": np.var, "sum":np.sum}
    
    # build rolling means for variables with input window days
    df_with_new_vars = pd.DataFrame(index=df.index)
    for dw in day_windows:
        for func_name in functions:
            temp_df = df[variables].rolling(dw).apply(func_names[func_name])
            temp_df.columns = [col + "_" + str(dw) + "_" + func_name for col in temp_df.columns]
            df_with_new_vars = df_with_new_vars.merge(temp_df, on="date")
    
    # drop rows with NaNs
    df_with_new_vars = df_with_new_vars.dropna()
    
    return df_with_new_vars

In [75]:
bam = add_supervised_target(df, 30)

In [80]:
bam

,purchases,invoices,revenue,streams,views,target
date,,,,,,
2017-11-28,3203,162,14139.14,1342,14948,181908.26
2017-11-29,3271,130,13396.92,1397,14135,168511.34
2017-11-30,2980,149,13250.07,1331,15560,155261.27
2017-12-01,2515,105,9517.35,1246,12180,147710.47
2017-12-02,370,31,1263.28,263,3101,149508.46
...,...,...,...,...,...,...
2019-07-26,1131,70,3631.94,688,7552,27536.85
2019-07-28,1192,40,2862.54,762,6227,24674.31
2019-07-29,1187,50,4316.05,770,6879,20358.26


In [73]:
bum = add_supervised_variables(df, ["purchases", "invoices"], ("mean", "std"))

In [77]:
bum.head()

,purchases_3_mean,invoices_3_mean,purchases_3_std,invoices_3_std,purchases_5_mean,invoices_5_mean,purchases_5_std,invoices_5_std,purchases_7_mean,invoices_7_mean,purchases_7_std,invoices_7_std
date,,,,,,,,,,,,
2017-12-04,1706.666667,80.333333,1014.366578,37.249907,2123.0,99.0,948.148933,39.354796,2441.285714,112.428571,946.429035,40.376291
2017-12-05,2389.666667,125.666667,368.823656,32.013886,2010.8,102.6,870.075261,44.279115,2329.285714,113.142857,894.632840,41.281018
2017-12-06,2570.666667,133.333333,181.608247,24.087802,2001.2,104.0,864.707442,44.443222,2214.428571,110.571429,814.368266,40.707518
2017-12-07,2438.666667,132.666667,20.531818,24.444949,2413.2,121.6,287.378078,25.389762,2135.857143,106.285714,761.532933,37.919894
2017-12-08,2146.333333,117.333333,427.595084,3.858612,2336.8,128.0,425.069124,19.778777,1996.857143,108.571429,768.411800,38.254251


In [78]:
pd.merge(bum, bam, on="date")
#bam.head()

,purchases_3_mean,invoices_3_mean,purchases_3_std,invoices_3_std,purchases_5_mean,invoices_5_mean,purchases_5_std,invoices_5_std,purchases_7_mean,invoices_7_mean,purchases_7_std,invoices_7_std,purchases,invoices,revenue,streams,views,target
date,,,,,,,,,,,,,,,,,,
2017-12-04,1706.666667,80.333333,1014.366578,37.249907,2123.0,99.0,948.148933,39.354796,2441.285714,112.428571,946.429035,40.376291,2826,121,13023.36,1334,12350,134443.37
2017-12-05,2389.666667,125.666667,368.823656,32.013886,2010.8,102.6,870.075261,44.279115,2329.285714,113.142857,894.632840,41.281018,2419,167,9358.97,1190,12474,129942.93
2017-12-06,2570.666667,133.333333,181.608247,24.087802,2001.2,104.0,864.707442,44.443222,2214.428571,110.571429,814.368266,40.707518,2467,112,11263.69,1307,10493,118679.24
2017-12-07,2438.666667,132.666667,20.531818,24.444949,2413.2,121.6,287.378078,25.389762,2135.857143,106.285714,761.532933,37.919894,2430,119,10816.89,1197,11688,109942.45
2017-12-08,2146.333333,117.333333,427.595084,3.858612,2336.8,128.0,425.069124,19.778777,1996.857143,108.571429,768.411800,38.254251,1542,121,4898.32,876,7922,119375.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-26,1222.333333,91.666667,114.636624,21.514853,1366.2,85.2,288.594802,19.405154,1319.285714,77.857143,260.371556,20.109902,1131,70,3631.94,688,7552,27536.85
2019-07-28,1235.666667,77.000000,107.803319,33.436507,1221.6,80.0,90.612582,26.161040,1303.714286,75.285714,264.230264,23.371009,1192,40,2862.54,762,6227,24674.31
2019-07-29,1170.000000,53.333333,27.652607,12.472191,1209.2,73.0,90.256080,28.467525,1315.714286,73.714286,256.640780,24.615409,1187,50,4316.05,770,6879,20358.26


In [41]:
cc = pd.DataFrame(index=df.index)
bu = df[["purchases", "invoices"]].rolling(3).mean()
#cc = pd.merge(cc, bu, on="date")
#bi = df[["purchases", "invoices"]].rolling(5).mean()
#cc = pd.merge(cc, bu, on="date")

In [45]:
[name + "_" + str(3) for name in bu.columns]

['purchases_3', 'invoices_3']

In [47]:
cc.merge(bu, on="date")

,purchases,invoices
date,,
2017-11-28,NaN,NaN
2017-11-29,NaN,NaN
2017-11-30,3151.333333,147.000000
2017-12-01,2922.000000,128.000000
2017-12-02,1955.000000,95.000000
...,...,...
2019-07-26,1222.333333,91.666667
2019-07-28,1235.666667,77.000000
2019-07-29,1170.000000,53.333333


In [43]:
bu

,purchases,invoices
date,,
2017-11-28,NaN,NaN
2017-11-29,NaN,NaN
2017-11-30,3151.333333,147.000000
2017-12-01,2922.000000,128.000000
2017-12-02,1955.000000,95.000000
...,...,...
2019-07-26,1222.333333,91.666667
2019-07-28,1235.666667,77.000000
2019-07-29,1170.000000,53.333333


In [39]:
cc

,purchases_x,invoices_x,purchases_y,invoices_y
date,,,,
2017-11-28,NaN,NaN,NaN,NaN
2017-11-29,NaN,NaN,NaN,NaN
2017-11-30,3151.333333,147.000000,3151.333333,147.000000
2017-12-01,2922.000000,128.000000,2922.000000,128.000000
2017-12-02,1955.000000,95.000000,1955.000000,95.000000
...,...,...,...,...
2019-07-26,1222.333333,91.666667,1222.333333,91.666667
2019-07-28,1235.666667,77.000000,1235.666667,77.000000
2019-07-29,1170.000000,53.333333,1170.000000,53.333333


In [26]:
for day in df.index[:10]:
    
    start = day + pd.Timedelta(days=1)
    end = start + pd.Timedelta(days=30)
    
    print(df["revenue"].loc[start:end].sum())

181908.26000000164
168511.3400000013
155261.27000000104
147710.47000000105
149508.460000001
144119.62000000104
134443.37000000084
129942.9300000008
118679.24000000073
109942.45000000068
